In [3]:
import math
import networkx as nx
import db, pathSearch
points = [[35.16487131336672, 136.9374775886536], [35.16378437005421, 136.9397842884064], [35.16297791885458, 136.9374775886536]]
y1, x1, y2, x2 = pathSearch.rectangleArea(points)
link, length = db.getRectangleRoadData(y1, x1, y2, x2)
G = pathSearch.linkToGraph(link, length)
length = dict(nx.all_pairs_dijkstra_path_length(G))
print(length['[35.1635255, 136.9400303]'])

{'[35.1635255, 136.9400303]': 0, '[35.1636073, 136.9394786]': 0.05096947, '[35.1634227, 136.9394362]': 0.071854803, '[35.1640066, 136.9395699]': 0.09613872300000001, '[35.1629381, 136.9393217]': 0.126735943, '[35.164399, 136.9396591]': 0.14051861300000001, '[35.1637556, 136.938429]': 0.14779575, '[35.1635797, 136.9383886]': 0.16769641700000001, '[35.1647606, 136.9397392]': 0.18138073300000002, '[35.1634256, 136.9383532]': 0.18513120000000002, '[35.1643158, 136.9402128]': 0.19169428600000002, '[35.1641528, 136.9385202]': 0.19273375, '[35.1630828, 136.9382744]': 0.22328948299999998, '[35.1651538, 136.9398408]': 0.226067333, '[35.1645473, 136.9386028]': 0.237238057, '[35.1639101, 136.9373535]': 0.24705967, '[35.1627501, 136.9381986]': 0.260920283, '[35.1649102, 136.9386832]': 0.27824724700000003, '[35.163582, 136.9372752]': 0.284656085, '[35.1643126, 136.9374495]': 0.291671634, '[35.1632437, 136.9371944]': 0.323082239, '[35.1647013, 136.937521]': 0.33537900400000004, '[35.1650675, 136.937

In [4]:
p, le,_ = pathSearch.travelingPath(points, link, length)
print(p)
print(le)
print(y1, x1, y2, x2)

[[35.1647013, 136.937521], [35.1643126, 136.9374495], [35.1641528, 136.9385202], [35.1640066, 136.9395699], [35.1640066, 136.9395699], [35.1641528, 136.9385202], [35.1637556, 136.938429], [35.1639101, 136.9373535], [35.163582, 136.9372752], [35.1632437, 136.9371944], [35.1629191, 136.9371148], [35.1629191, 136.9371148], [35.1632437, 136.9371944], [35.163582, 136.9372752], [35.1639101, 136.9373535], [35.1643126, 136.9374495], [35.1647013, 136.937521]]
0.79436472
35.16487131336672 136.9374775886536 35.16297791885458 136.9397842884064


In [2]:
G = pathSearch.linkToGraph(link, length)
print(list(G.nodes))

['[35.1624723, 136.9403682]', '[35.1626084, 136.9392426]', '[35.1657816, 136.9399919]', '[35.1651538, 136.9398408]', '[35.1653085, 136.9387857]', '[35.1652278, 136.9393279]', '[35.1634444, 136.940595]', '[35.1635255, 136.9400303]', '[35.1636073, 136.9394786]', '[35.1645473, 136.9386028]', '[35.164399, 136.9396591]', '[35.1643158, 136.9402128]', '[35.1642346, 136.9407811]', '[35.1628586, 136.9398764]', '[35.1649102, 136.9386832]', '[35.1647606, 136.9397392]', '[35.1658611, 136.9394589]', '[35.1634194, 136.936091]', '[35.1632437, 136.9371944]', '[35.1630828, 136.9382744]', '[35.1629381, 136.9393217]', '[35.1627768, 136.9404377]', '[35.1641528, 136.9385202]', '[35.1637556, 136.938429]', '[35.1635797, 136.9383886]', '[35.1634256, 136.9383532]', '[35.1627501, 136.9381986]', '[35.1640066, 136.9395699]', '[35.1634227, 136.9394362]', '[35.1618272, 136.9390696]', '[35.1625422, 136.9370223]', '[35.1629191, 136.9371148]', '[35.163582, 136.9372752]', '[35.1639101, 136.9373535]', '[35.1643126, 136.

In [3]:
tsp = ['[35.1647013, 136.937521]', '[35.1640066, 136.9395699]', '[35.1629191, 136.9371148]', '[35.1647013, 136.937521]']
p = tsp[0]
rp = p.strip("[]")
print(p)
print(rp)

[35.1647013, 136.937521]
35.1647013, 136.937521


In [2]:
import csv
import networkx as nx
import db, pathSearch

# データを用意
points = [[35.16966936723668, 136.90265178680423], [35.15224248922456, 136.95080280303958]]
y1, x1, y2, x2 = pathSearch.rectangleArea(points)
link, length = db.getRectangleRoadData(y1, x1, y2, x2)
G = pathSearch.linkToGraph(link, length)
length = dict(nx.all_pairs_dijkstra_path_length(G))
# CSVファイルにデータを書き込む
with open('data.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter='\t')  # タブを区切り文字として指定
    for u in length.keys():
        for v in length[u].keys():
            data = [u, v, length[u][v]]
            writer.writerow(data)


In [2]:
import csv

dic = {}
with open('data.csv', 'r', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    l = [row for row in reader]
    for elem in l:
        if elem[0] in dic.keys():
            dic[elem[0]][elem[1]] = elem[2]
        else:
            dic[elem[0]] = {elem[1] : elem[2]}
            
print(dic['[35.1635255, 136.9400303]']['[35.1636073, 136.9394786]'])

0.05096947


In [1]:
nodes = []
def dist():
    return
def swap():
    return

while(True):
    Flag = True
    for p in nodes:
        for q in nodes:
            d1 = dist(p, q)
            d2 = dist(p+1, q+1)
            d3 = dist(p, p+1)
            d4 = dist(q, q+1)
            if d1+d2 < d3+d4:
                swap(p, q)
                Flag = False
    if Flag:
        break


False


In [39]:
import pathSearch, db
import random, time
import numpy as np
from scipy import stats

N = 50
moveDist = 1000
ite = 20

def generate_random_points_in_rectangle(rectangle, n):
    x_min, y_min = min(rectangle[0][0], rectangle[1][0]), min(rectangle[0][1], rectangle[1][1])
    x_max, y_max = max(rectangle[0][0], rectangle[1][0]), max(rectangle[0][1], rectangle[1][1])
    random_points = [[random.uniform(x_min, x_max), random.uniform(y_min, y_max)] for _ in range(n)]
    return random_points

coordinates = [[35.18441016255083, 136.9190883636475], [35.15573127979675, 136.97341918945315]]

sum_len_old = 0
sum_len_walk_old = 0
sum_len_new = 0
sum_len_walk_new = 0
sum_time_old = 0
sum_time_new = 0
count_old = 0
count_new = 0
t_test_old = []
t_test_old_walk = []
t_test_new = []
t_test_new_walk = []
for i in range(ite):
    points = generate_random_points_in_rectangle(coordinates, N)
    y1, x1, y2, x2 = pathSearch.rectangleArea(points)
    try:
        link, length = db.getRectangleRoadData(y1, x1, y2, x2, 1.25)

        start = time.perf_counter()
        _, len_old, _, _, _, len_walk_old = pathSearch.sharedRidePath(points, link, length, moveDist, "type4", len_dic)
        end = time.perf_counter()
        sum_time_old += end - start
        sum_len_old += len_old
        sum_len_walk_old += len_walk_old
        count_old += 1
        t_test_old.append(len_old)
        t_test_old_walk.append(len_walk_old)

        start = time.perf_counter()
        _, len_new, _, _, _, len_walk_new = pathSearch.sharedRidePath(points, link, length, moveDist, "type2", len_dic)
        end = time.perf_counter()
        sum_time_new += end - start
        sum_len_new += len_new
        sum_len_walk_new += len_walk_new
        count_new += 1
        t_test_new.append(len_new)
        t_test_new_walk.append(len_walk_new)

    except:
        continue
ave_len_old = sum_len_old / count_old
ave_len_walk_old = sum_len_walk_old / count_old
ave_len_new = sum_len_new / count_new
ave_len_walk_new = sum_len_walk_new / count_new
ratio = (ave_len_old - ave_len_new) / ave_len_old
person = (ave_len_walk_old - ave_len_walk_new) / N
ave_time_old = sum_time_old / count_old
ave_time_new = sum_time_new / count_new
print(ave_len_old)
print(ave_len_walk_old)
print(ave_len_new)
print(ave_len_walk_new)
print(ratio*100)
print(person*1000)
print(count_old,count_new)
print(ave_time_old, ave_time_new)

if count_new == count_old:
    print(stats.ttest_rel(t_test_old, t_test_new))
    print(stats.ttest_rel(t_test_old_walk, t_test_new_walk))


12.903891148337491
34.600611640900006
12.6813294042125
28.26231253823125
1.72476458121445
126.76598205337514
16 16
15.986185343750321 15.560754406249998
TtestResult(statistic=2.2665087032973, pvalue=0.03864164135360939, df=15)
TtestResult(statistic=13.583185623456872, pvalue=7.80986847888958e-10, df=15)


In [40]:
import networkx as nx
import pathSearch, db
coordinates = [[35.18441016255083, 136.9190883636475], [35.15573127979675, 136.97341918945315]]
y1, x1, y2, x2 = pathSearch.rectangleArea(coordinates)
link, length = db.getRectangleRoadData(y1, x1, y2, x2)
G = pathSearch.linkToGraph(link, length)
len_dic = dict(nx.all_pairs_dijkstra_path_length(G))
print("---data loaded---")

---data loaded---


In [47]:

import math
import random
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def generate_random_points_in_rectangle(rectangle, n):
    x_min, y_min = min(rectangle[0][0], rectangle[1][0]), min(rectangle[0][1], rectangle[1][1])
    x_max, y_max = max(rectangle[0][0], rectangle[1][0]), max(rectangle[0][1], rectangle[1][1])
    random_points = [[random.uniform(x_min, x_max), random.uniform(y_min, y_max)] for _ in range(n)]
    return random_points

def nearestNode(p, link):
    dist = float('inf')
    nearestNode = None
    for line in link:
        for point in line:
            d = math.sqrt((p[0] - point[0])**2 + (p[1] - point[1])**2)
            if d < dist:
                dist = d
                nearestNode = point
    return nearestNode

def len_SP(G, node1, node2, len_dic):
    try:
        return len_dic[node1][node2]
    except KeyError:
        return nx.dijkstra_path_length(G, node1, node2)
    
def connectGraph(G):
    if not nx.algorithms.components.is_connected(G):
        components = list(nx.algorithms.components.connected_components(G))
        for i in range(len(components) - 1):
            if not nx.algorithms.components.is_connected(G):
                component1 = components[i]
                component2 = components[i + 1]
                node1 = next(iter(component1))
                node2 = next(iter(component2))
                G.add_weighted_edges_from([(node1, node2, float('inf'))])

def linkToGraph(link, length):
    edges = []
    for i in range(len(link)):
        edges.append((str(link[i][0]), str(link[i][1]), length[i]))
    G = nx.Graph()
    G.add_weighted_edges_from(edges)
    return G

positions = []
points = generate_random_points_in_rectangle(coordinates, 30)
points[0] = [35.17010416043001, 136.9457817077637]

def create_data_model(points, link, length, len_dic):
    G = linkToGraph(link, length)
    connectGraph(G)
    positions = []
    for p in points:
        positions.append(str(nearestNode(p, link)))

    data = {}
    # DMはintegerである必要がある
    DM = []
    for p in positions:
        d = []
        for q in positions:
            d.append(int(len_SP(G, p, q, len_dic)*1000))
        DM.append(d)

    data["distance_matrix"] = DM
    data["num_vehicles"] = 4
    data["depot"] = 0
    print(DM)

    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")

def main():
    data = create_data_model(points, link, length, len_dic)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        30000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # Extract routes into a matrix form
        routes_matrix = []
        for vehicle_id in range(data['num_vehicles']):
            route = []
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                node = manager.IndexToNode(index)
                route.append(node)
                index = solution.Value(routing.NextVar(index))
            # Add the depot (ending point)
            route.append(manager.IndexToNode(index))
            routes_matrix.append(route)
        if routes_matrix:
            for i, route in enumerate(routes_matrix):
                print(f"Route for vehicle {i}: {route}")

    else:
        print("No solution found!")

if __name__ == "__main__":
    main()

[[0, 1629, 339, 1830, 252, 1595, 3130, 1394, 3109, 1187, 1524, 1088, 2366, 1857, 2263, 1726, 2295, 1652, 1541, 3203, 1687, 2154, 973, 1780, 2408, 681, 101, 2532, 2794, 1082], [1629, 0, 1733, 562, 1376, 567, 2743, 2777, 4503, 2442, 2819, 2687, 3136, 3076, 2759, 2496, 3632, 2798, 629, 4091, 1356, 1507, 872, 2793, 2172, 1763, 1721, 2151, 2557, 2460], [339, 1733, 0, 2000, 423, 1479, 2793, 1107, 2822, 1406, 1188, 1303, 2651, 2076, 2551, 2011, 1987, 1316, 1711, 3422, 1857, 2324, 1143, 1999, 2578, 367, 395, 2702, 2964, 796], [1830, 562, 2000, 0, 1577, 1077, 3250, 3064, 4815, 2424, 3125, 2785, 2754, 2868, 2289, 2151, 3925, 3124, 555, 3620, 857, 1008, 999, 2570, 1672, 2070, 1922, 1652, 2058, 2746], [252, 1376, 423, 1577, 0, 1342, 3094, 1486, 3237, 1316, 1548, 1341, 2381, 1986, 2172, 1741, 2347, 1676, 1289, 3321, 1473, 1938, 728, 1909, 2194, 594, 344, 2318, 2580, 1168], [1595, 567, 1479, 1077, 1342, 0, 2176, 2401, 3935, 2659, 2396, 2684, 3467, 3329, 3122, 2827, 3197, 2360, 1083, 4422, 1810, 1961

In [1]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")


if __name__ == "__main__":
    main()

Objective: 177500
Route for vehicle 0:
 0 ->  9 ->  10 ->  2 ->  6 ->  5 -> 0
Distance of the route: 1712m

Route for vehicle 1:
 0 ->  16 ->  14 ->  8 -> 0
Distance of the route: 1484m

Route for vehicle 2:
 0 ->  7 ->  1 ->  4 ->  3 -> 0
Distance of the route: 1552m

Route for vehicle 3:
 0 ->  13 ->  15 ->  11 ->  12 -> 0
Distance of the route: 1552m

Maximum of the route distances: 1712m
